# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
%matplotlib inline
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


/home/student/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/student/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/student/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/student/anaconda3/envs/te

## Env setup

## Object detection imports
Here are the imports from the object detection module.

In [2]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [3]:
# What model to download.
MODEL_NAME = 'faster10k_rcnn101_inference_graph'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME +'/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('kitchen_data', 'Kitchen_label_map.pbtxt')

## Load a (frozen) Tensorflow model into memory.

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [5]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [6]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [7]:
from glob import glob
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images/KitchenSHPhone/'
#TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 5) ]
TEST_IMAGE_PATHS = glob ("test_images/KitchenSHPhone/*.jpg") # your image path

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


In [8]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [9]:
import os
from tqdm import tqdm
from sys import exit
import argparse
import cv2
from textwrap import dedent
import xml.etree.ElementTree as ET
import xml.etree.ElementTree
from lxml import etree

def create_xml_annotation(img_width, img_height,ymin_l,xmin_l,ymax_l,xmax_l,image_path,class_name,images_dir):
    print("\n" + "Creating PASCAL VOC XML Files for Class:")
    
    #######
    annotation = etree.Element("annotation")
    
    folder = etree.Element("folder")
    folder.text = os.path.basename(os.getcwd())
    annotation.append(folder)  

    
    filename_xml = etree.Element("filename")
    filename_xml.text = os.path.basename(image_path)
    annotation.append(filename_xml)  
    
    path = etree.Element("path")
    path.text = os.path.join(images_dir,filenamestr)
    annotation.append(path)

    
    source = etree.Element("source")
    annotation.append(source)

    database = etree.Element("database")
    database.text = "Unknown"
    source.append(database)

    size = etree.Element("size")
    annotation.append(size)

    width = etree.Element("width")
    height = etree.Element("height")
    depth = etree.Element("depth")
    
    # Reading Curent Image for Attributes...
    os.chdir(maindir)
    img = cv2.imread(image_path)

    width.text = str(img.shape[1])
    height.text = str(img.shape[0])
    depth.text = str(img.shape[2])

    size.append(width)
    size.append(height)
    size.append(depth)

    segmented = etree.Element("segmented")
    segmented.text = "0"
    annotation.append(segmented)
    
    obj = etree.Element("object")
    annotation.append(obj)

    name = etree.Element("name")
    name.text = class_name
    obj.append(name)

    pose = etree.Element("pose")
    pose.text = "Unspecified"
    obj.append(pose)

    truncated = etree.Element("truncated")
    truncated.text = "0"
    obj.append(truncated)

    difficult = etree.Element("difficult")
    difficult.text = "0"
    obj.append(difficult)

    bndbox = etree.Element("bndbox")
    obj.append(bndbox)

    xmin = etree.Element("xmin")
    xmin.text = str(int(xmin_l*img_width))
    bndbox.append(xmin)

    ymin = etree.Element("ymin")
    ymin.text = str(int(ymin_l*img_height))
    bndbox.append(ymin)

    xmax = etree.Element("xmax")
    xmax.text = str(int(xmax_l*img_width))
    bndbox.append(xmax)

    ymax = etree.Element("ymax")
    ymax.text = str(int(ymax_l*img_height))
    bndbox.append(ymax)
    os.chdir(images_dir)
    os.chdir("..")
    if not os.path.exists('Annotations'):
        os.makedirs('Annotations')
    os.chdir('Annotations')
    s = etree.tostring(annotation, pretty_print=True)
    
    #filename_str = str.split(filenamestr, ".")[0]
    with open(fname_str , 'wb') as f:
        f.write(s)
        f.close()
    print("created xml file")
   



In [10]:
%matplotlib inline
from matplotlib import pyplot as plt
import scipy.misc
import time
import cv2
for image_path in TEST_IMAGE_PATHS:
  maindir=os.getcwd()  
  image = Image.open(image_path)
  width, height = image.size
  print(os.path.basename(image_path))
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  # Visualization of the results of a detection.
  #print(output_dict)
  iamge1,ymin,xmin,ymax,xmax,class_name=vis_util.return_visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=5)
  #plt.figure(figsize=IMAGE_SIZE)
  #plt.imshow(image_np)
  images_dir=os.path.join(os.getcwd(),PATH_TO_TEST_IMAGES_DIR)
  os.chdir(images_dir)
  os.chdir("..")
  if not os.path.exists('Annotations'):
        os.makedirs('Annotations')
  os.chdir('Annotations')
    ## Checking if XML already exists ...
  filenamestr = os.path.basename(image_path)
  fname_str = str.split(filenamestr, ".")[0]
  fname_str = fname_str+'.xml'
  if not os.path.isfile(fname_str):
        create_xml_annotation(width, height,ymin,xmin,ymax,xmax,image_path,class_name,images_dir)
  elif os.path.isfile(fname_str):
    doc = xml.dom.minidom.parse(fname_str)
    ann = doc.firstChild
    verified = ann.getAttribute("verified")
    if not verified == "yes":
        create_xml_annotation(width, height,ymin,xmin,ymax,xmax,image_path,class_name,images_dir)
              
  os.chdir(maindir)
  #cv2.imwrite('outfile-{}.jpg'.format(int(time.time())),cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

image14.jpg

Creating PASCAL VOC XML Files for Class:
created xml file
image19.jpg


KeyboardInterrupt: 